In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
def get_title_and_url_from_aidb(soup):
    data = soup.select('div[id="post_list"] > div[class="post_item clearfix"]')
    # BeautifulSoupを使用してデータを抽出
    titles = []
    urls = []
    release_dates = []
    for item in data:
        release_date = item.select_one('time[class="entry-date updated"]').text
        release_dates.append(release_date)

        title = item.select_one('div[class="post_info"] > h3[class="title"] > a').text
        titles.append(title)

        url = item.select_one('div[class="post_info"] > h3[class="title"] > a')["href"]
        urls.append(url)

    # データフレームを作成
    df = pd.DataFrame({
        'release_date': release_dates,
        'title': titles,
        'url': urls
    })

    return df

In [3]:
df = pd.DataFrame(columns=["release_date", "title", "url"])

for i in range(1,20):
    # url定義
    url = f"https://aiboom.net/page/{i}"

    # urlへアクセス
    sleep(1)
    response = requests.get(url)

    # html解析
    soup = BeautifulSoup(response.content, 'html.parser')

    # titleと記事urlの抽出
    tmp_db = get_title_and_url_from_aidb(soup)

    df = pd.concat([df, tmp_db], axis=0, ignore_index=True)


In [4]:
df

,release_date,title,url
0,2023/10/27,GPT-4などLLMのコード生成能力にデバッグ機能を追加する『SELF-DEBUGGING（...,https://aiboom.net/archives/57709
1,2023/10/26,LLMに非線形的な思考を与えてCoTを上回る性能を引き出す手法『IEP』と実行プロンプト C...,https://aiboom.net/archives/57628
2,2023/10/25,"Natureなどの論文約4,800本でGPT-4による査読能力が検証され、「LLMは査読にも...",https://aiboom.net/archives/57449
3,2023/10/24,DALL-E 3はどうしてユーザーの意図を正確に汲み取ることができるのか？OpenAIが論文で発表,https://aiboom.net/archives/57310
4,2023/10/23,LLMにベートーヴェンなど特定の人物の行動や感情を模倣させる、イタコのような技術『Chara...,https://aiboom.net/archives/57223
...,...,...,...
147,2021/11/19,【Facebook AI】2人競技における選手の動きをAIでシミュレーション（AI×スポーツ...,https://aiboom.net/archives/49274
148,2021/11/1,ボクシング・パンチを機械学習 AIによるトレーニングコーチの仕事【AI×スポーツ】（論文解説）,https://aiboom.net/archives/49248
149,2021/10/12,AIで稲の病気を検出 害虫の被害を防止する自動化テクノロジー【AI×農業】（論文解説）,https://aiboom.net/archives/48905
150,2021/10/5,1枚のスケッチから世に無い新しい写真を作り出すことが可能に（AI論文解説）,https://aiboom.net/archives/48923


In [5]:
df.to_excel("../data/aidb/index.xlsx", index=False)